# Gradient descent for other functions

Gradient descent works well for other functions, even if they are non-convex; gradient descent, if it converges, converges to a local minimum. Lets look at the function $$f(a,b) = \sin(a)\cos(b).$$ We would like to find parameters $(a,b)$ that minimize the function $f$. 

The gradients are:
$$\frac{\partial f}{\partial a} = \cos(a)\cos(b),$$
$$\frac{\partial f}{\partial b} = -\sin(a)\sin(b).$$

In [ ]:
# Code source: Sebastian Curi and Andreas Krause.

# Python Notebook Commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# Numerical Libraries
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 5)

# IPython Libraries
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual


# Custom Libraries
from utilities.load_data import polynomial_data
from utilities import plot_helpers
from utilities.regressors import LinearRegressor


In [ ]:
class sincos(object):
    def __init__(self):
        pass

    def get_number_samples(self):
        return 0

    def loss(self, w, *args):
        return np.sin(w[0]) * np.cos(w[1])

    def gradient(self, w, *args):
        return np.array([np.cos(w[0]) * np.cos(w[1]), -np.sin(w[0]) * np.sin(w[1])])

In [ ]:
def gradient_descent(w0, loss_function, opts=dict()):
    """
    Vanilla Gradient descent algorithm. 
    
    w0: is the initial guess
    loss_function: is the loss function you want to optimize. It should have the gradient method. 
    opts: a dictionary with the algorithm parameters 
    """
    w = w0
    dim = w0.size
    
    # Parse the options. 
    eta = opts.get('eta0', 0.01)
    eta0 = eta
    n_iter = opts.get('n_iter', 10)
    learning_rate_scheduling = opts.get('learning_rate_scheduling', None) 
    
    # Trajectory of iterates w
    trajectory = np.zeros((n_iter + 1, dim))
    trajectory[0, :] = w
    
    # Needed for learning rate schedule. 
    f_val = loss_function.loss(w)
    f_old = f_val
    grad_sum = 0
    
    for it in range(n_iter):
        # Compute Gradient
        gradient = loss_function.gradient(w)
        grad_sum += np.sum(np.square(gradient)) # For AdaGrad
        
        # Update learning rate.
        learning_rate_opts = {'learning_rate_scheduling': learning_rate_scheduling,
                              'eta0': eta0,
                              'it': it,
                              'f_increased': (f_val > f_old),
                              'grad_sum': grad_sum}
        eta = compute_learning_rate(eta, learning_rate_opts)  # Note it is recursive but eta0 is passed in the options.
        
        # Perform gradient step.
        w = w - eta * gradient
    
        # Save weights.
        trajectory[it + 1, :] = w
        
        # Compute new cost.
        f_old = f_val
        f_val = loss_function.loss(w)

    return trajectory

def compute_learning_rate(eta, opts=dict()):
    learning_rate_scheduling = opts.get('learning_rate_scheduling', None)
    eta0 = opts.get('eta0', eta)
    f_increased = opts.get('f_increased', False)
    grad_sum = opts.get('grad_sum', 0)
    it = opts.get('it', 0)
    if learning_rate_scheduling == None:
        eta = eta0  # keep it constant. 
    elif learning_rate_scheduling == 'Annealing':
        eta = eta0 / np.power(it + 1, 0.6)
    elif learning_rate_scheduling == 'Bold driver':
        eta = (eta / 5) if (f_increased) else (eta * 1.1)
    elif learning_rate_scheduling == 'AdaGrad':
        eta = eta0 / np.sqrt(grad_sum)
    elif learning_rate_scheduling == 'Annealing2':
        eta = min([eta0, 100. / (it + 1.)])
    else:
        raise ValueError('Learning rate scheduling {} not understood'.format(method))
    return eta


In [ ]:
eta_widget = ipywidgets.FloatSlider(value=4, min=1e-1, max=10, step=1e-1, readout_format='.1f', 
                                    description='Learning rate:', style={'description_width': 'initial'}, 
                                    continuous_update=False)
n_iter_widget = ipywidgets.IntSlider(value=10, min=5, max=50, step=1, description='Number of iterations:',
                                     style={'description_width': 'initial'}, continuous_update=False)

lr_widget = ipywidgets.RadioButtons(options=['Bold driver', 'AdaGrad', 'Annealing', 'None'], value='None',
                                    description='Learning rate heuristics:', style={'description_width': 'initial'})

a0_widget = ipywidgets.FloatSlider(value=-0.9, min=-3, max=3, step=.1, readout_format='.1f', description='w_0:',
                                   style={'description_width': 'initial'}, continuous_update=False)

b0_widget = ipywidgets.FloatSlider(value=1.1, min=-3, max=3, step=.1, readout_format='.1f', description='w_1:',
                                   style={'description_width': 'initial'}, continuous_update=False)
def optimize_sincos(eta, n_iter, learning_rate_scheduling, a0, b0):
    loss_function = sincos()
    w0 = np.array([a0, b0])
    
    if learning_rate_scheduling == 'None':
        learning_rate_scheduling = None

    opts = {'eta0': eta,
            'n_iter': n_iter,
            'learning_rate_scheduling': learning_rate_scheduling
            }
    trajectory = gradient_descent(w0, loss_function, opts)

    contourplot = plt.subplot(111)
    dataplot = None
    contour_opts = {'x_label': '$w_0$', 'y_label': '$w_1$', 'title': 'Weight trajectory', 'legend': False}
    plot_opts = {'contour_opts': contour_opts}

    plot_helpers.linear_regression_progression(np.array([]), np.array([]), trajectory, np.array([]), 
                                               loss_function.loss,
                                               contourplot, dataplot, options=plot_opts)


interact_manual(optimize_sincos, eta=eta_widget, n_iter=n_iter_widget, learning_rate_scheduling=lr_widget,
                a0=a0_widget, b0=b0_widget);